In [1]:
print("OK")

OK


In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [4]:
os.environ['GOOGLE_API_KEY']=GOOGLE_API_KEY

In [5]:
from langchain.document_loaders import PyPDFLoader

In [6]:
%pwd

'c:\\Users\\Admin\\Interview_Question_Creator_Project\\research'

In [7]:
%cd ..

c:\Users\Admin\Interview_Question_Creator_Project


c:\Users\Admin\anaconda3\envs\interview\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
%pwd

'c:\\Users\\Admin\\Interview_Question_Creator_Project'

In [10]:
file_path="data/SDG Resource Document_Targets Overview.pdf"
loader=PyPDFLoader(file_path)
data=loader.load()

In [11]:
data

[Document(page_content='4th SDG Youth Summer Camp – SDG Resource Document  \nThe 2030 Agenda for Sustainable Development’s 17 \nSustainable Development Goals (SDGs)    \nGoal: This document enables 4th SDG Youth Summer Camp participants to i) get to \nknow the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify \ntargets of most interest to participants, and iv) identify synergies between the SDGs \nand chosen target(s).  \n \n \nGoal 1. End poverty in all its forms everywhere   \nTarget 1.1  By 2030, eradicate extreme poverty  for all people everywhere, currently measured \nas people living on less than $1.25 a day  \nTarget 1.2  By 2030, reduce at least by half the proportion  of men, women and  children of all \nages living in poverty  in all its dimensions according to national definitions  \nTarget 1.3  Implement nationally appropriate social protection systems and measures for \nall, including floors, and by 2030 achieve substantial coverage of the p

In [12]:
len(data)

19

In [13]:
question_gen=""
for page in data:
    question_gen=question_gen+page.page_content

In [14]:
question_gen

'4th SDG Youth Summer Camp – SDG Resource Document  \nThe 2030 Agenda for Sustainable Development’s 17 \nSustainable Development Goals (SDGs)    \nGoal: This document enables 4th SDG Youth Summer Camp participants to i) get to \nknow the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify \ntargets of most interest to participants, and iv) identify synergies between the SDGs \nand chosen target(s).  \n \n \nGoal 1. End poverty in all its forms everywhere   \nTarget 1.1  By 2030, eradicate extreme poverty  for all people everywhere, currently measured \nas people living on less than $1.25 a day  \nTarget 1.2  By 2030, reduce at least by half the proportion  of men, women and  children of all \nages living in poverty  in all its dimensions according to national definitions  \nTarget 1.3  Implement nationally appropriate social protection systems and measures for \nall, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable 

In [16]:
from langchain.text_splitter import TokenTextSplitter

In [17]:
splitter_ques_gen=TokenTextSplitter(model_name="gpt-3.5-turbo",chunk_size=1000,chunk_overlap=100)

In [18]:
chunk_ques_gen=splitter_ques_gen.split_text(question_gen)

In [19]:
chunk_ques_gen

['4th SDG Youth Summer Camp – SDG Resource Document  \nThe 2030 Agenda for Sustainable Development’s 17 \nSustainable Development Goals (SDGs)    \nGoal: This document enables 4th SDG Youth Summer Camp participants to i) get to \nknow the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify \ntargets of most interest to participants, and iv) identify synergies between the SDGs \nand chosen target(s).  \n \n \nGoal 1. End poverty in all its forms everywhere   \nTarget 1.1  By 2030, eradicate extreme poverty  for all people everywhere, currently measured \nas people living on less than $1.25 a day  \nTarget 1.2  By 2030, reduce at least by half the proportion  of men, women and  children of all \nages living in poverty  in all its dimensions according to national definitions  \nTarget 1.3  Implement nationally appropriate social protection systems and measures for \nall, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable

In [20]:
len(chunk_ques_gen)

9

In [21]:
type(chunk_ques_gen)

list

In [22]:
type(chunk_ques_gen[0])

str

In [23]:
from langchain.docstore.document import Document

In [24]:
document_ques_gen=[Document(page_content=t) for t in chunk_ques_gen]

In [25]:
document_ques_gen

[Document(page_content='4th SDG Youth Summer Camp – SDG Resource Document  \nThe 2030 Agenda for Sustainable Development’s 17 \nSustainable Development Goals (SDGs)    \nGoal: This document enables 4th SDG Youth Summer Camp participants to i) get to \nknow the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify \ntargets of most interest to participants, and iv) identify synergies between the SDGs \nand chosen target(s).  \n \n \nGoal 1. End poverty in all its forms everywhere   \nTarget 1.1  By 2030, eradicate extreme poverty  for all people everywhere, currently measured \nas people living on less than $1.25 a day  \nTarget 1.2  By 2030, reduce at least by half the proportion  of men, women and  children of all \nages living in poverty  in all its dimensions according to national definitions  \nTarget 1.3  Implement nationally appropriate social protection systems and measures for \nall, including floors, and by 2030 achieve substantial coverage of the p

In [26]:
type(document_ques_gen[0])

langchain_core.documents.base.Document

In [28]:
splitter_ans_gen=TokenTextSplitter(model_name="gpt-3.5-turbo",chunk_size=800,chunk_overlap=80)

In [29]:
document_answer_gen=splitter_ans_gen.split_documents(document_ques_gen)

In [30]:
len(document_answer_gen)

17

In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain.chatmodels import ChatOpenAI
from langchain_core.messages import HumanMessage

In [34]:
model=ChatGoogleGenerativeAI(model='gemini-pro',temperature=0.4,google_api_key=GOOGLE_API_KEY)

In [35]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [36]:
from langchain.prompts import PromptTemplate

In [37]:
PROMP_QUESTIONS=PromptTemplate(template=prompt_template,input_variables=['text'])

In [38]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [39]:
REFINE_PROMPT_QUESTIONS=PromptTemplate(template=refine_template,input_variables=['existing_answer','text'])

In [40]:
from langchain.chains.summarize import load_summarize_chain

In [41]:
ques_gen_chain=load_summarize_chain(llm=model,chain_type='refine',verbose=True,question_prompt=PROMP_QUESTIONS,refine_prompt=REFINE_PROMPT_QUESTIONS)

In [42]:
ques=ques_gen_chain.run(document_ques_gen)
print(ques)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
4th SDG Youth Summer Camp – SDG Resource Document  
The 2030 Agenda for Sustainable Development’s 17 
Sustainable Development Goals (SDGs)    
Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to 
know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify 
targets of most interest to participants, and iv) identify synergies between the SDGs 
and chosen target(s).  
 
 
Goal 1. End poverty in all its forms everywhere   
Target 1.1  By 2030, eradicate extreme poverty  for all people everywhere, currently measured 
as people living on less than $1.25 a day  
Target 1.2  By 2030, reduce at least by

In [43]:
from langchain_community.embeddings import HuggingFaceEmbeddings
#from langchain.embeddings.openai import OpenAIEmbeddings

In [44]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs={'device':'cpu'})

c:\Users\Admin\anaconda3\envs\interview\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [45]:
from langchain.vectorstores import FAISS

In [46]:
vector_store=FAISS.from_documents(document_answer_gen,embeddings)

In [57]:
llm_answer_gen=ChatGoogleGenerativeAI(model='gemini-pro',temperature=0.1,google_api_key=GOOGLE_API_KEY,verbose=True,n=10)

In [48]:
ques

'**Refined Questions:**\n\n**Question 1:**\nWhat is the primary purpose of the 4th SDG Youth Summer Camp?\n\n**Question 2:**\nHow many Sustainable Development Goals (SDGs) are outlined in the 2030 Agenda for Sustainable Development?\n\n**Question 3:**\nWhat is the target date for eradicating extreme poverty under Goal 1?\n\n**Question 4:**\nWhich target under Goal 1 aims to ensure equal rights to economic resources for all men and women?\n\n**Question 5:**\nWhat is the target date for ending hunger and ensuring access to safe and nutritious food for all under Goal 2?\n\n**Question 6:**\nWhich target under Goal 2 focuses on doubling the agricultural productivity and incomes of small-scale food producers?\n\n**Question 7:**\nWhat is the target date for maintaining the genetic diversity of seeds and cultivated plants under Goal 2?\n\n**Question 8:**\nWhich target under Goal 2 aims to correct and prevent trade restrictions and distortions in world agricultural markets?\n\n**Question 9:**\n

In [49]:
ques_list=ques.split("\n")

In [50]:
ques_list

['**Refined Questions:**',
 '',
 '**Question 1:**',
 'What is the primary purpose of the 4th SDG Youth Summer Camp?',
 '',
 '**Question 2:**',
 'How many Sustainable Development Goals (SDGs) are outlined in the 2030 Agenda for Sustainable Development?',
 '',
 '**Question 3:**',
 'What is the target date for eradicating extreme poverty under Goal 1?',
 '',
 '**Question 4:**',
 'Which target under Goal 1 aims to ensure equal rights to economic resources for all men and women?',
 '',
 '**Question 5:**',
 'What is the target date for ending hunger and ensuring access to safe and nutritious food for all under Goal 2?',
 '',
 '**Question 6:**',
 'Which target under Goal 2 focuses on doubling the agricultural productivity and incomes of small-scale food producers?',
 '',
 '**Question 7:**',
 'What is the target date for maintaining the genetic diversity of seeds and cultivated plants under Goal 2?',
 '',
 '**Question 8:**',
 'Which target under Goal 2 aims to correct and prevent trade restric

In [58]:
from langchain.chains import RetrievalQA

In [59]:
answer_gen_chain=RetrievalQA.from_chain_type(llm=llm_answer_gen,chain_type="stuff",retriever=vector_store.as_retriever())

In [61]:
answer_gen_chain.invoke(ques_list[0])

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Request contains an invalid argument.

In [55]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_gen_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  **Refined Questions:**


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Developer instruction is not enabled for models/gemini-pro